# Extract Title and Author

In [28]:
import pandas as pd
refers = pd.read_csv('reference.csv', header=None)
refers = refers[0].tolist()
print(refers)

['\tBC Cancer Protocol Summary BRAVPTRAT Page 7 of 7 Activated: 1 Aug 2017 Revised: 1 Apr 2018 (Typo, institutional name, tests)\t', "\tWarning: The information contained in these documents are a statement of consensus of BC Cancer professionals regarding their views of currently accepted approaches to treatment. Any clinician seeking to apply or consult these documents is expected to use independent medical judgement in the context of individual clinical circumstances to determine any patient's care or treatment. Use of these documents is at your own risk and is subject to BC Cancer's terms of use available at www.bccancer.bc.ca/legal.htm\t", '\t\t', '\t\t', '\t\t', '\t\t', '\t1. Gradishar WJ, et al. Phase III Trial of nanoparticle albumin-bound pacliitaxel compared with polethylated castor oil-based pacliitaxel in women with breast cancer J Clin Oncol 2005;23:7794-7803 2. Abraxis Oncology. ABRAXANE® product monograph. Richmond Hill, Ontario; 26 June 2006.\t', '\tBC Cancer Agency Prot

In [62]:
pattern = r"\.;? (([^\. ,]+ ){4,}[^\. ,]+)\."
import re
re_p = re.compile(pattern)
titles = []
for refer in refers:
    ress = re_p.findall(refer)
    for res in ress:
        titles.append(res[0])
    
#s = "\t1. Cameron D, et al. Lapatinib plus capecitabine in women with HER-2 positive advanced breast cancer: final survival analysis of phase III randomized trial. Oncologist 2010;15:924-34. 2. Geyer CE, et al. Lapatinib plus capecitabine for HER2-positive advanced breast cancer. N Engl J Med 2006;355:2733-43.\t', '\t\t', '\t\t', '\t\t', '\tBC Cancer Agency Protocol Summary BRAJLHRHT\t"
#res = re_p.findall(s)
#print(res)
#[('Lapatinib plus capecitabine in women with HER-2 positive advanced breast cancer: final survival analysis of phase III randomized trial', 'randomized '), ('Lapatinib plus capecitabine for HER2-positive advanced breast cancer', 'breast ')]

In [63]:
print(len(titles))

5383


# Title to PMID

In [71]:
import requests
import time
import json
import urllib.parse
import editdistance


def thesametitle(t1, t2):
    edit_dist = editdistance.eval(t1, t2)
    if edit_dist<=4:
        return True
    else:
        return False
    
def pmids2pmidTotitle(pmids):
    pmids_ = ','.join(pmids) 
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&'+\
          f'id={pmids_}' + '&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08'
    
    pmidTotitle = {}
    
    try:
        rs = requests.get(url)
        #print(url)
        time.sleep(0.1)
        try:
            rs_dict = json.loads(rs.text)
            
        except json.decoder.JSONDecodeError as e:
            print(f'json decode error, ulr={url}')
            print(e)
            return None
        if 'result' in rs_dict:
            for pmid in pmids:
                title = rs_dict['result'][pmid]['title']
                pmidTotitle.update({pmid:title})
            return pmidTotitle
            
    
    except requests.exceptions.RequestException as e:
        print(f'requests.get({url}) results in this error:')
        print(e)

    return None
    


# 'Comparative genomics of the eukaryotes'
# should return 10731134
def title2pmid(title):
    title_ = urllib.parse.quote_plus(title)
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&'+\
          f'term={title_}' + '&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08'
    try:
        rs = requests.get(url)
        print(url)
        time.sleep(0.1)
        try:
            rs_dict = json.loads(rs.text)
            
        except json.decoder.JSONDecodeError as e:
            print(f'json decode error, ulr={url}')
            print(e)
            return None
        if 'esearchresult' in rs_dict:
            if 'idlist' in rs_dict['esearchresult']:
                pmids = rs_dict['esearchresult']['idlist']
                pmidTotitle = pmids2pmidTotitle(pmids)
                if pmidTotitle is not None:
                    for pmid in pmids:
                        if thesametitle(pmidTotitle[pmid],title):
                            return pmid
    
    except requests.exceptions.RequestException as e:
        print(f'requests.get({url}) results in this error:')
        print(e)

    return None

In [60]:
## test
#title = 'Comparative genomics of the eukaryotes'
#print(title2pmid(title))

# get pdf_title_pmid.csv

In [76]:
pdf_title_pmid={'Title':[], 'PMID':[]}
cnt = 0
import time
ss = time.time()

for title in titles:
    pmid = title2pmid(title)
    cnt+=1
    if cnt%100 ==0:
        print(cnt)
        print(time.time()-ss)
    if pmid is not None:
        pdf_title_pmid['Title'].append(title)
        pdf_title_pmid['PMID'].append(pmid)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Phase+III+Trial+of+nanoparticle+albumin-bound+pacliitaxel+compared+with+polethylated+castor+oil-based+pacliitaxel+in+women+with+breast+cancer+J+Clin+Oncol+2005%3B23%3A7794-7803+2&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b6005

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Pharmacokinetics+%28PK%29+and+safety+of+trastuzumab+%28Herceptin+%EF%9B%9A+%29+when+administered+every+three+weeks+to+women+with+metastatic+breast+cancer&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Clinical+cardiac+tolerability+of+trastuzumab&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Febrile+neutropenia+rates+with+adjuvant+docetaxel+and+cyclophophamide+chemotherapy+in+early+

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=NCCN+Clinical+Practice+Guidelines+in+Oncology%3A+Breast+cancer+%28V&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Cancer+Drug+Manual&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Extended+adjuvant+bisphosphonate+treatment+over+five+years+in+early+breast+cancer+does+not+improve+disease-free+and+overall+survival+compared+to+two+years+of+treatment%3A+Phase+III+data+f

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=N+Engl+J+Med+2011%3B364%3A1728-378&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Docetaxel+in+patients+with+metastatic+breast+cance

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=A+randomized+trial+of+letrozole+in+postmenopausal+women+after+five+years+of+tamoxifen+therapy+for+early-stage+breast+cancer&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=J+Clin+Oncol+1994%3B12+%3A1630-1638&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Any+clinician+seeking+to+apply+or+consult+these+documents+is+expected+to+use+independent+medical+judgement+in+the+context+of+individual+clinical+circumstances+to+determine+any+patient%27s+care+or+treatment&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Benefit+of+a+high-dose+epirubicin+regimen+in+adjuvant+chemotherapy+for+node-+positive+breast+cancer+patients+with+poor+prognostic+f

...

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Randomized+phase+II+study+of+two+schedules+of+topotecan+in+previously+treated+patients+with+ovarian+cancer%3A+a+National+Cancer+Institute+of+Canada+Clinical+Trials+Group+study&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Topotecan%3A+a+review+of+its+efficacy+in+small+cell+lung+cancer&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Topotecan+treatment+of+adults+with+primary+malignant+glioma&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=New+developments+in+the+treatment+of+acute+myeloid+leukemia%3A+focus+on+topotecan&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Survival+of+patients+with+high+grade+glioma+treated+with+intrathecal+thiotriethylenephosphoramide+for+ependymal+or+leptomeningeal+gliomatosis&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Pharmacokinetics+following+intraventricular+administration+of+chemotherapy+in+patients+with+neoplastic+meningitis&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=DRUGDEX%C2%AE+Evaluations+%28database+on+the+Internet%29&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Cytochrome+P450+isozymes+3A4+and+2B6+are+involved+in+the+in+vitro+human+metabolism+of+thiotepa+to+TEPA&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://euti

5200
4444.44989323616
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Breast+Tumour+Group%3B+15+March+2001&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=DRUGDEX%C2%AE+Evaluations+%28database+on+the+Internet%29&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Breast+Tumour+Group+Pharmacist%3B+14+July+2010&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Breast+Tumour+Group%3B+20+July+2010&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=%28SCNAUSEA%29+Guidelines+for+Prevention+and+Treatment+of+Chemotherapy-induced+Nausea+and+Vomiting+in+Ad

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Effect+of+gastric+pH+on+the+relative+oral+bioavailability+and+pharmacokinetics+of+temozolomide&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Temozolomide+stability+in+extemporaneously+compounded+oral+suspensions&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Gastrointestinal+Tumour+Group&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Neuro-Oncology+Tumour+Group&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Neuro-Oncology+Tumour+Group&field=title&format=json&api_key=b9056d083d23b60058e

requests.get(https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Lymphoma+and+Myeloma+Tumour+Group%3B+31+October+2016&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08) results in this error:
HTTPSConnectionPool(host='eutils.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /entrez/eutils/esearch.fcgi?db=pubmed&term=BC+Cancer+Agency+Lymphoma+and+Myeloma+Tumour+Group%3B+31+October+2016&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4b86106a58>: Failed to establish a new connection: [Errno -2] Name or service not known'))
requests.get(https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Integrated+safety+profile+of+single-agent+carfilzomib%3A+experience+from+526+patients+enrolled+in+4+phase+II+clinical+studies&field=title&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08) results in 

In [77]:
print(len(pdf_title_pmid['Title']))
import pandas as pd
pd.DataFrame(pdf_title_pmid).to_csv('pdf_title_pmid.csv',index=False)

1246
